In [7]:
import pandas as pd
import yfinance as yf
import calendar
import requests
import ffn
import numpy as np
from tqdm import tqdm

In [2]:
nifty_50_symbols ={'ADANIENT',
 'ADANIPORTS',
 'AMBUJACEM',
 'APOLLOHOSP',
 'ASIANPAINT',
 'AUROPHARMA',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJAJFINSV',
 'BAJFINANCE',
 'BHARTIARTL',
 'BOSCHLTD',
 'BPCL',
 'BRITANNIA',
 'CIPLA',
 'COALINDIA',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GAIL',
 'GRASIM',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDALCO',
 'HINDPETRO',
 'HINDUNILVR',
 'IBULHSGFIN',
 'ICICIBANK',
 'INDUSINDBK',
 'INFRATEL',
 'INFY',
 'IOC',
 'ITC',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'LUPIN',
 'M&M',
 'MARUTI',
 'NESTLEIND',
 'NTPC',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBILIFE',
 'SBIN',
 'SHREECEM',
 'SUNPHARMA',
 'TATACONSUM',
 'TATAMOTORS',
 'TATASTEEL',
 'TCS',
 'TECHM',
 'TITAN',
 'ULTRACEMCO',
 'UPL',
 'VEDL',
 'WIPRO',
 'YESBANK',
 'ZEEL'}

In [5]:
# above are the all the list of stocks in nifty from 2018 - count 63
#https://www.niftyindices.com/Indices_-_Market_Capitalisation_and_Weightage/indices_dataFeb2022.zip"
#downloaded from above link by replacing month and year

In [6]:
result = pd.DataFrame(columns=[
    "Symbol","Date","year","month","move","full_move","open_high",
    "next_day_open","close_to_next_day_open","nextdayopen_high",
    "nextdaymove","nextday_full_move","Volume","Close",'next_day_close'
    
])

In [8]:
result = pd.DataFrame()

In [9]:
for name in tqdm(nifty_50_symbols):
    try:
        df = yf.download(name+".NS",interval='1d',progress=False,start="2015-01-01")
        df['year'] = df.index.year
        df['month'] = df.index.month
        df = df.reset_index()
        df['full_move'] = (df['High']/df['Low']-1)*100
        df['move'] = (df['Close']/df['Open']-1)*100
        df['open_high'] = (df['High']/df['Open']-1)*100
        df['next_day_open'] = df.Open.shift(-1)
        df['close_to_next_day_open'] = (df['next_day_open']/df['Close']-1)*100
        df['nextdayopen_high'] = df.open_high.shift(-1)
        df['nextdaymove'] = df.move.shift(-1)
        df['nextday_full_move'] = df.full_move.shift(-1)
        df['next_day_close'] = df.Close.shift(-1)
        idx = df[df.move>5].index.values
        df.loc[:,'Symbol'] = name
        result = pd.concat([result, df[["Symbol","Date","year","month","move","full_move","open_high",
        "next_day_open","close_to_next_day_open","nextdayopen_high",
        "nextdaymove","nextday_full_move","Volume","Close",'next_day_close']]])
        #break
    except:
        pass

 83%|█████████████████████████████████████████████████████████▊            | 52/63 [00:23<00:04,  2.68it/s]


1 Failed download:
- INFRATEL.NS: No data found, symbol may be delisted


100%|██████████████████████████████████████████████████████████████████████| 63/63 [00:28<00:00,  2.25it/s]


In [10]:
#this will give list of stocked moved by 5% that day
result = result[(result.move>5)].reset_index()
result.shape

(1538, 16)

In [11]:
#consider the dates where we have only 5 or lesser trades
dates = result.groupby(['Date'])['Symbol'].count().sort_values(ascending=False)[result.groupby(['Date'])['Symbol'].count().sort_values(ascending=False)<=5].index

In [12]:
dates = list(dates)
len(dates)

685

In [13]:
result = result[(result.Date.isin(dates))].reset_index()

In [14]:
#invest 100000 on each stock(very few days we invest 5lakshs rest days on average we will have only 1 or 2 trades)
result['qty'] = np.int64(100000/result['Close'])

In [15]:
result['Investment'] = result['qty']*result['Close']

In [16]:
#considering i square it off at next day open
result["Profit"] = result['next_day_open']*result['qty'] - result['Close']*result['qty']

In [17]:
trades_per_day = result.groupby(['Date']).Symbol.count().reset_index()
capital_per_day = result.groupby(['Date']).Investment.sum().reset_index()
profit_per_day = result.groupby(['Date']).Profit.sum().reset_index()

In [18]:
final = pd.merge(pd.merge(trades_per_day,capital_per_day),profit_per_day)

In [21]:
final.columns = ['Date','No Of Symbols Bought','Investment','Profit']
final['return'] = (final.Profit/final.Investment)*100

In [22]:
final.head()

,Date,No Of Symbols Bought,Investment,Profit,return
0,2015-01-02,1,99912.801270,364.000000,0.364318
1,2015-01-07,1,99787.500000,717.498779,0.719027
2,2015-01-09,1,99428.997192,333.502808,0.335418
3,2015-01-14,1,99632.277100,1690.919678,1.697161
4,2015-01-15,2,198397.601562,1808.796875,0.911703


In [23]:
final['Profit'].sum()

497646.89084243774

In [24]:
final['Date'] = pd.to_datetime(final['Date'])
final = final.sort_values(['Date']).reset_index()
final['year'] = final.Date.dt.year
final['month'] = final.Date.dt.month

In [25]:
final.groupby(['year']).Profit.sum()

year
2015     44440.352829
2016     26849.395905
2017     29279.291924
2018     18233.485596
2019     97068.162334
2020    161162.660727
2021     73633.952606
2022     46979.588923
Name: Profit, dtype: float64

In [28]:
avg_investment = final.Investment.mean()
avg_investment

160077.11877268075

In [29]:
final.loc[0,'Profit'] = final.loc[0,'Profit'] +200000

In [30]:
profit = final.groupby(['Date']).Profit.sum()
profit = profit.cumsum().copy()
stats = profit.calc_stats()

/Users/santhoshk/Library/Python/3.8/lib/python/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/santhoshk/Library/Python/3.8/lib/python/site-packages/ffn/core.py:2299: RuntimeWarning: divide by zero encountered in divide
  res = np.divide(er.mean(), std)


In [31]:
stats.display()

Stats for Profit from 2015-01-02 00:00:00 - 2022-12-29 00:00:00
Annual risk-free rate considered: 0.00%
Summary:
Total Return      Sharpe  CAGR    Max Drawdown
--------------  --------  ------  --------------
248.19%             4.28  16.90%  -4.22%

Annualized Returns:
mtd    3m     6m     ytd    1y     3y      5y      10y    incep.
-----  -----  -----  -----  -----  ------  ------  -----  --------
1.16%  1.08%  3.28%  7.22%  7.22%  19.12%  18.41%  -      16.90%

Periodic:
        daily    monthly    yearly
------  -------  ---------  --------
sharpe  4.28     2.12       1.32
mean    46.59%   15.42%     16.72%
vol     10.88%   7.28%      12.66%
skew    0.44     2.45       1.24
kurt    10.65    9.66       -0.04
best    4.70%    12.60%     38.75%
worst   -4.22%   -2.43%     6.07%

Drawdowns:
max     avg       # days
------  ------  --------
-4.22%  -0.66%     20.76

Misc:
---------------  -------
avg. up month    1.82%
avg. down month  -0.72%
up year %        100.00%
12m up %         96

In [33]:
result

,level_0,index,Symbol,Date,year,month,move,full_move,open_high,next_day_open,close_to_next_day_open,nextdayopen_high,nextdaymove,nextday_full_move,Volume,Close,next_day_close,qty,Investment,Profit
0,0,136,INFY,2015-07-21,2015,7,7.502418,11.014493,11.014493,552.500000,-0.687550,0.633484,-0.167419,3.283330,38310112,556.325012,551.575012,179,99582.177185,-684.677185
1,1,255,INFY,2016-01-14,2016,1,6.285178,9.546168,8.348968,574.000000,1.323919,1.393728,-0.705573,2.325168,30227006,566.500000,569.950012,176,99704.000000,1320.000000
2,2,466,INFY,2016-11-25,2016,11,5.056692,5.862753,5.862753,489.649994,0.184142,0.423775,0.045952,1.323927,12631380,488.750000,489.875000,204,99705.000000,183.598755
3,6,1286,INFY,2020-03-24,2020,3,12.988503,15.162840,15.162840,597.000000,1.220755,6.365159,1.541041,12.250307,20027097,589.799988,606.200012,169,99676.197937,1216.802063
4,7,1348,INFY,2020-06-26,2020,6,5.350605,5.859151,5.829339,735.400024,-1.710771,1.264616,-0.496332,1.999731,26030458,748.200012,731.750000,133,99510.601624,-1702.398376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,1533,1501,SBIN,2021-02-02,2021,2,5.746034,7.111109,7.111109,333.750000,0.195135,1.842695,0.659180,4.327811,121434166,333.100006,335.950012,300,99930.001831,194.998169
1100,1534,1503,SBIN,2021-02-04,2021,2,6.190195,8.124432,7.057418,387.000000,8.983383,5.516797,1.576229,6.023630,145203439,355.100006,393.100006,281,99783.101715,8963.898285
1101,1535,1569,SBIN,2021-05-17,2021,5,5.131727,5.926033,5.708018,390.000000,1.801095,0.653843,-1.987179,3.289041,106555796,383.100006,382.250000,261,99989.101593,1800.898407
1102,1536,1744,SBIN,2022-01-25,2022,1,5.030617,5.828221,5.612245,503.549988,-2.156813,5.421505,5.044191,5.494826,21791739,514.650024,528.950012,194,99842.104736,-2153.407104
